In [1]:
import selenium 
from selenium import webdriver
import pandas as pd
import time
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import requests
import os
import urllib.request
from bs4 import BeautifulSoup as bs

1. Write a python program which searches all the product under a particular product from
www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user
input is ‘guitar’. Then search for guitars.
2. In the above question, now scrape the following details of each product listed in first 3 pages
of your search results and save it in a dataframe and csv. In case if any product has less than 3
pages in search results then scrape all the products available under that product vertical.
Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings",
"Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and
“Product URL”. In case, if any of the details are missing for any of the product then replace it
by “-“.

In [2]:
#importing driver
driver=webdriver.Chrome(r'C:\Users\Alby\Desktop\chromedriver\chromedriver.exe')

In [457]:
#getting url
url='https://www.amazon.in/'
driver.get(url)

#search bar
search_bar=driver.find_element_by_id('twotabsearchtextbox')
search_bar.send_keys('headphones')

#search button
search_button=driver.find_element_by_id('nav-search-submit-button')
search_button.click()
time.sleep(2)

#product name
Product_names=[]
total_rating=[]
Price=[]
Del_date=[]
Prod_url=[]

for i in range(2):
    #product names
    product_tags=driver.find_elements_by_xpath("//span[@class='a-size-medium a-color-base a-text-normal']")
    time.sleep(3)
    for names in product_tags:
        if names.text is None:
            Product_names.append("-")
        else:
            Product_names.append(names.text)
    
    #rating
    tot_rating=driver.find_elements_by_xpath("//span[@class='a-size-base']")
    for i in tot_rating:
         if i.text is None:
            total_rating.append("-")
         else:
            total_rating.append(i.text)
        
    #total price
    tot_price=driver.find_elements_by_xpath("//span[@class='a-price-whole']")
    for price in tot_price:
        if price.text is None:
            Price.append("-")
        else:
            Price.append(price.text)
    
    #expected delivery
    ex_del=driver.find_elements_by_xpath("//span[@class='a-text-bold']")
    
    for day in ex_del:
        if day.text is None:
            Del_date.append("-")
        else:
            Del_date.append(day.text)
        
    prod_url=driver.find_elements_by_xpath("//a[@class='a-link-normal a-text-normal']")
    for i in prod_url:
        Prod_url.append(i.get_attribute('href'))

    
    #next button
    time.sleep(5)
    next=driver.find_element_by_xpath("//li[@class='a-last']//a")#scraping the list of buttons from the page
    next_button=driver.get(next.get_attribute('href'))
#getting brand names
Brand=[]
for names in Product_names:
    Brand.append(names.split()[0])

In [458]:
#for details inside each urls

Availability=[]
Rating=[]
other_feature=[]

#accessing each urls
for item in Prod_url:
    time.sleep(1)
    driver.get(item)
   
#accessing availbilty
    avail=driver.find_elements_by_id("availability")
    for status in avail:
        Availability.append(status.text.replace('\n'," "))
    
    #accessing raing
    rating=driver.find_elements_by_xpath("//span[@class='a-size-medium a-color-base']")
    for j in rating:
        if j.text is None:
            Rating.append("-")
        else:
            Rating.append(j.text)

#others
    other_features=driver.find_elements_by_xpath("//td[@class='a-span9']//span")
    for feat in other_features:
        if feat.text is None:
            other_feature.append("-")
        else:
            other_feature.append(feat.text)
    other_feature.append("+") #adding + sign so that it is easier to split later

    #partitioning the features list
others = []
temp = []
for item in other_feature:
    temp.append(item)
    if item == '+':
        others.append(temp)
        temp = []
if temp:
    others.append(temp)
        

In [460]:
Amazon_headphones=pd.DataFrame({})
Amazon_headphones["Product_name"]=Product_names[0:45]
Amazon_headphones["Brand"]=Brand[0:45]
Amazon_headphones["price"]=Price[0:45]
Amazon_headphones["Rating"]=Rating[0:45]
Amazon_headphones["Availability"]=Availability[0:45]
Amazon_headphones["Delivery_by"]=Del_date[0:45]
Amazon_headphones["Numberof_ratings"]=total_rating[0:45]
Amazon_headphones["Other_details"]=others[0:45]
Amazon_headphones["url"]=Prod_url[0:45]
Amazon_headphones.to_csv('Amazon_headphones.csv')
Amazon_headphones.head()

,Product_name,Brand,price,Rating,Availability,Delivery_by,Numberof_ratings,Other_details,url
0,Marshall Major III Bluetooth Wireless On-Ear H...,Marshall,"7,499",4.4 out of 5,Only 1 left in stock.,"Thursday, July 8","8,070","[Marshall, Black, Bluetooth, Major 3 BT, On Ea...",https://www.amazon.in/gp/slredirect/picassoRed...
1,Fire-Boltt Blast 1400 Over -Ear Bluetooth Wire...,Fire-Boltt,"2,499",4.1 out of 5,In stock.,"Thursday, July 8","4,900","[Fire-Boltt, Black, Wireless, BH1400, Over Ear...",https://www.amazon.in/gp/slredirect/picassoRed...
2,boAt Bassheads 900 On Ear Wired Headphones(Car...,boAt,799,4.2 out of 5,In stock.,"Thursday, July 8","43,278","[BoAt, Carbon Black, Wired, BassHeads, On Ear, +]",https://www.amazon.in/Boat-BassHeads-900-Wired...
3,boAt Rockerz 450 On-Ear Headphones with 15 Hou...,boAt,"1,199",4.1 out of 5,In stock.,"Monday, July 12","33,894","[BoAt, Luscious Black, Wireless, Rockerz 450, ...",https://www.amazon.in/Rockerz-450-Wireless-Blu...
4,Zebronics Zeb-Thunder Wireless BT Headphone Co...,Zebronics,699,3.9 out of 5,In stock.,"Friday, July 9","21,812","[ZEBRONICS, Blue, Bluetooth, Wired, ZEB-Thunde...",https://www.amazon.in/Zebronics-Zeb-Thunder-Bl...


### 3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [82]:
url='https://images.google.com/'
driver.get(url)
time.sleep(7)

#search bar
search_bar=driver.find_element_by_xpath('/html/body/div[2]/div[2]/div/form/div[1]/div[1]/div[1]/div/div[2]/input')
search_bar.send_keys('fruits')

In [83]:
#search_button
search_button=driver.find_element_by_xpath("//span[@class='z1asCe MZy1Rb']")
search_button.click()

In [84]:
#scolling
for i in range(1,10):
    time.sleep(2)
    driver.execute_script("window.scrollBy(0, 4000)") 

In [85]:
#fruits images
count=0
kk=(r'C:\Users\Alby\Desktop\projects\fliprobo\Selenium\Images\Fruits')
elements = driver.find_elements_by_xpath('//img[contains(@class,"rg_i Q4LuWd")]')
for i in elements[:100]:
    src = i.get_attribute('src')
    try:
        if src != None:
            src  = str(src)
            count+=1
            
            urllib.request.urlretrieve(src, os.path.join(kk,'fruits'+str(count)+'.jpg'))
        else:
            raise TypeError
    except TypeError:
        pass

In [87]:
#scrapping cars images
url='https://images.google.com/'
driver.get(url)
time.sleep(7)

#search bar
search_bar=driver.find_element_by_xpath('/html/body/div[2]/div[2]/div/form/div[1]/div[1]/div[1]/div/div[2]/input')
search_bar.send_keys('Cars')
time.sleep(2)
#search_button
search_button=driver.find_element_by_xpath("//span[@class='z1asCe MZy1Rb']")
search_button.click()
time.sleep(2)
#scolling
for i in range(1,10):
    time.sleep(2)
    driver.execute_script("window.scrollBy(0, 4000)") 
    
#cars images
count=0
car=(r'C:\Users\Alby\Desktop\projects\fliprobo\Selenium\Images\Cars')
elements = driver.find_elements_by_xpath('//img[contains(@class,"rg_i Q4LuWd")]')
for i in elements[:100]:
    src = i.get_attribute('src')
    try:
        if src != None:
            src  = str(src)
            count+=1
            
            urllib.request.urlretrieve(src, os.path.join(car,'cars'+str(count)+'.jpg'))
        else:
            raise TypeError
    except TypeError:
        pass
    

In [89]:
#scrapping machine learning images
url='https://images.google.com/'
driver.get(url)
time.sleep(7)

#search bar
search_bar=driver.find_element_by_xpath('/html/body/div[2]/div[2]/div/form/div[1]/div[1]/div[1]/div/div[2]/input')
search_bar.send_keys('Machine learning')
time.sleep(2)
#search_button
search_button=driver.find_element_by_xpath("//span[@class='z1asCe MZy1Rb']")
search_button.click()
time.sleep(2)

#scolling
for i in range(1,10):
    time.sleep(2)
    driver.execute_script("window.scrollBy(0, 4000)") 
    
#machine learning images
count=0
Machine_learning=(r'C:\Users\Alby\Desktop\projects\fliprobo\Selenium\Images\Machine learning')
elements = driver.find_elements_by_xpath('//img[contains(@class,"rg_i Q4LuWd")]')
for i in elements[:100]:
    src = i.get_attribute('src')
    try:
        if src != None:
            src  = str(src)
            count+=1
            
            urllib.request.urlretrieve(src, os.path.join(Machine_learning,'ML'+str(count)+'.jpg'))
        else:
            raise TypeError
    except TypeError:
        pass

### 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. In case if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [301]:
#getting url
url='https://www.flipkart.com/'
driver.get(url)
time.sleep(2)

#close login prompt
try:
    btn=driver.find_element_by_xpath("//button[@class='_2KpZ6l _2doB4z']")
    btn.click()
except NoSuchElementException:
    pass
time.sleep(2)


#sarch bar
search_bar=driver.find_element_by_xpath("/html/body/div[1]/div/div[1]/div[1]/div[2]/div[2]/form/div/div/input")


#passing key word
search_bar.send_keys("pixel 4a")

#search_button
search_button=driver.find_element_by_xpath("//button[@class='L0Z3Pu']")
search_button.click()

In [ ]:
#lists to be scraped
phone_name=[]
Prod_urls=[]
Price=[]
name_tags=driver.find_elements_by_xpath("//div[@class='_4rR01T']")
for name in name_tags:
    phone_name.append(name.text)
price_tags=driver.find_elements_by_xpath("//div[@class='_30jeq3 _1_WHN1']")
for price in price_tags:
    Price.append(price.text)

#taking in urls   
urls=driver.find_elements_by_xpath("//a[@class='_1fQZEK']")
for i in urls:
    Prod_urls.append(i.get_attribute('href'))


#extracting brand names
Brand=[]
for i in phone_name:
    Brand.append(i.split()[0])

In [ ]:
#accessing details inside each products
Ram=[]
Colour=[]
ROM=[]
Prim_Cam=[]
Display=[]
Resolution=[]
Battery=[]
Rows=[]
value=[]
for url in Prod_urls:
    driver.get(url)
    time.sleep(4)
    
    #clicking read more button
    read_more=driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']")
    read_more.click()
    time.sleep(3)
    
    #saving table
    table_rows=driver.find_elements_by_xpath("//tr[@class='_1s_Smc row']")
    for i in table_rows:
        Rows.append(i.text)
        
    for name in table_rows:
        tex=name.find_element_by_xpath("td[1]").text 

        #Color
        if tex=='Color':
            
            Colour.append(name.find_element_by_xpath("td[2]").text)
        #Extracting Display Size
        if tex=='Display Size':
            Display.append(name.find_element_by_xpath("td[2]").text)

        #Resolution
        if tex=='Resolution':
             Resolution.append(name.find_element_by_xpath("td[2]").text)

        #Processor        
        if tex=='Processor Type':
            Processor.append(name.find_element_by_xpath("td[2]").text)

        #Cores
        if tex=='Processor Core':
            Cores.append(name.find_element_by_xpath("td[2]").text)

        #ROM
        if tex=='Internal Storage':
            ROM.append(name.find_element_by_xpath("td[2]").text)

        #RAM Memory
        if tex=='RAM':
            Ram.append(name.find_element_by_xpath("td[2]").text)

        #Extracting Primary Camera Details
        if tex=='Primary Camera':
            Prim_Cam.append(name.find_element_by_xpath("td[2]").text)
        # Extracting Battery Capacity
        if tex=='Battery Capacity':
            Battery.append(name.find_element_by_xpath("td[2]").text)

In [360]:
Rows=[]
Sec_cam=[]
Processor=[]
Cores=[]

for url in Prod_urls:
    soup1=[]
    driver.get(url)
    time.sleep(2)
    page_source = driver.page_source
    soup = bs(page_source, 'lxml')
    soup=soup.find_all('td',class_='_1hKmbr col col-3-12')
    for i in soup:
        soup1.append(i.text)
    
    #clicking read more button
    read_more=driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']")
    read_more.click()
    time.sleep(3)
    
    #saving table
    table_rows=driver.find_elements_by_xpath("//tr[@class='_1s_Smc row']")
    if 'Secondary Camera' in soup1:
        for name in table_rows:
            tes=name.find_element_by_xpath("td[1]").text
            if tes=='Secondary Camera':
                Sec_cam.append(name.find_element_by_xpath("td[2]").text)
    else:
        Sec_cam.append("-")
        
    if 'Processor Type' in soup1:
        for name in table_rows:
            tes=name.find_element_by_xpath("td[1]").text
            if tes=='Processor Type':
                Processor.append(name.find_element_by_xpath("td[2]").text)
    else:
        Processor.append("-")
        
    if 'Processor Core' in soup1:
        for name in table_rows:
            tes=name.find_element_by_xpath("td[1]").text
            if tes=='Processor Core':
                Cores.append(name.find_element_by_xpath("td[2]").text)
    else:
        Cores.append("-")           

In [362]:
flipkart=pd.DataFrame({})
flipkart['Phone_name']=phone_name
flipkart['Price']=Price
flipkart['Brand']=Brand
flipkart['Colour']=Colour
flipkart['Ram']=Ram
flipkart['ROM']=ROM
flipkart['Prim_Cam']=Prim_Cam
flipkart['Sec_cam']=Sec_cam
flipkart['Display']=Display
flipkart['Resolution']=Resolution
flipkart['Processor']=Processor
flipkart['Cores']=Cores
flipkart['Battery']=Battery
flipkart['Prod_urls']=Prod_urls
flipkart.to_csv('flipkart.csv')
flipkart.head()

,Phone_name,Price,Brand,Colour,Ram,ROM,Prim_Cam,Sec_cam,Display,Resolution,Processor,Cores,Battery,Prod_urls
0,"Google Pixel 4a (Just Black, 128 GB)","₹31,999",Google,Just Black,6 GB,128 GB,12.2MP Rear Camera,8MP Front Camera,14.76 cm (5.81 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 730G,Octa Core,3140 mAh,https://www.flipkart.com/google-pixel-4a-just-...
1,"REDMI Note 9 (Shadow Black, 64 GB)","₹10,999",REDMI,Shadow Black,4 GB,64 GB,48MP + 8MP + 2MP + 2MP,13MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G85,Octa Core,5020 mAh,https://www.flipkart.com/redmi-note-9-shadow-b...
2,"REDMI 9 Prime (Mint Green, 64 GB)","₹9,999",REDMI,Mint Green,4 GB,64 GB,13MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G80,Octa Core,5020 mAh,https://www.flipkart.com/redmi-9-prime-mint-gr...
3,"REDMI Note 9 (Pebble Grey, 64 GB)","₹10,999",REDMI,Pebble Grey,4 GB,64 GB,48MP + 8MP + 2MP + 2MP,13MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G85,Octa Core,5020 mAh,https://www.flipkart.com/redmi-note-9-pebble-g...
4,"REDMI Note 9 (Arctic White, 64 GB)","₹10,999",REDMI,Arctic White,4 GB,64 GB,48MP + 8MP + 2MP + 2MP,13MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G85,Octa Core,5020 mAh,https://www.flipkart.com/redmi-note-9-arctic-w...


### 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [482]:
#url
driver.get("https://maps.google.com")
time.sleep(2)

#accessing search bar

search_bar=driver.find_element_by_xpath("/html/body/jsl/div[3]/div[9]/div[3]/div[1]/div[1]/div[1]/div[2]/form/div/div[3]/div/input[1]")
search_bar.send_keys("Mumbai")
time.sleep(3)

#accessing search button

search_button=driver.find_element_by_xpath("//*[@id='searchbox-searchbutton']")
search_button.click()
time.sleep(2)


#getting the url
curr_url=driver.current_url
url_list=curr_url.split('/')
for i in url_list:
    if '@' in i:
        text=i
lat_long=text.split(',')
Latitude=lat_long[0].replace('@',"")
Longitude=lat_long[1]

print("The latitude:",Latitude,"\nThe Longitude:",Longitude)


The latitude: 19.0820391 
The Longitude: 72.6009774


### 6. Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 –September 20) from trak.in

In [485]:
#getting url
driver.get('https://trak.in/')
time.sleep(3)

#clicking on funding details
funding=driver.find_element_by_xpath("//*[@id='menu-item-51510']/a")
funding.click()

In [599]:
#accessing each table on monthly base
table_july=driver.find_elements_by_xpath('//table[@class="tablepress tablepress-id-48 dataTable no-footer"]')
table_aug=driver.find_elements_by_xpath('//table[@class="tablepress tablepress-id-49 dataTable no-footer"]')
table_sep=driver.find_elements_by_xpath('//table[@class="tablepress tablepress-id-50 dataTable no-footer"]')
Date=[]
Company=[]
Industry=[]
Location=[]
Subvertical=[]
Investor=[]
Investment_type=[]
Amount=[]


#Extracting from the July 2020 table
for i in table_july:
    date=i.find_elements_by_class_name("column-2")
    c_name=i.find_elements_by_class_name("column-3")
    Ind=i.find_elements_by_class_name("column-4")
    subvert=i.find_elements_by_class_name("column-5")
    loc=i.find_elements_by_class_name("column-6")
    investor=i.find_elements_by_class_name("column-7")
    invest_type=i.find_elements_by_class_name("column-8")
    amount=i.find_elements_by_class_name("column-9")
    
for day in date:
    Date.append(day.text)
for company in c_name:
    Company.append(company.text)
for j in Ind:
    Industry.append(j.text)
for k in subvert:
    Subvertical.append(k.text)
for l in loc:
    Location.append(l.text)
for m in investor:
    Investor.append(m.text)
for n in invest_type:
    Investment_type.append(n.text)
for o in amount:
    Amount.append(o.text)
    
#Extracting from the august 2020 table
for i in table_aug:
    date=i.find_elements_by_class_name("column-2")
    c_name=i.find_elements_by_class_name("column-3")
    Ind=i.find_elements_by_class_name("column-4")
    subvert=i.find_elements_by_class_name("column-5")
    loc=i.find_elements_by_class_name("column-6")
    investor=i.find_elements_by_class_name("column-7")
    invest_type=i.find_elements_by_class_name("column-8")
    amount=i.find_elements_by_class_name("column-9")
    
for day in date:
    Date.append(day.text)
for company in c_name:
    Company.append(company.text)
for j in Ind:
    Industry.append(j.text)
for k in subvert:
    Subvertical.append(k.text)
for l in loc:
    Location.append(l.text)
for m in investor:
    Investor.append(m.text)
for n in invest_type:
    Investment_type.append(n.text)
for o in amount:
    Amount.append(o.text)
    
    
#Extracting from the Sep 2020 table    
for i in table_sep:
    date=i.find_elements_by_class_name("column-2")
    c_name=i.find_elements_by_class_name("column-3")
    Ind=i.find_elements_by_class_name("column-4")
    subvert=i.find_elements_by_class_name("column-5")
    loc=i.find_elements_by_class_name("column-6")
    investor=i.find_elements_by_class_name("column-7")
    invest_type=i.find_elements_by_class_name("column-8")
    amount=i.find_elements_by_class_name("column-9")
    
for day in date:
    Date.append(day.text)
for company in c_name:
    Company.append(company.text)
for j in Ind:
    Industry.append(j.text)
for k in subvert:
    Subvertical.append(k.text)
for l in loc:
    Location.append(l.text)
for m in investor:
    Investor.append(m.text)
for n in invest_type:
    Investment_type.append(n.text)
for o in amount:
    Amount.append(o.text)    

In [600]:
Trak=pd.DataFrame({})
Trak['Date']=Date
Trak['Company']=Company
Trak['Industry']=Industry
Trak['Location']=Location
Trak['Subvertical']=Subvertical
Trak['Investor']=Investor
Trak['Investment_type']=Investment_type
Trak['Amount']=Amount
Trak

,Date,Company,Industry,Location,Subvertical,Investor,Investment_type,Amount
0,,,,,,,,
1,15/07/2020,Flipkart,E-commerce,Bangalore,E-commerce,Walmart Inc,M&A,"1,200,000,000"
2,16/07/2020,Vedantu,EduTech,Bangalore,Online Tutoring,Coatue Management,Series D,"100,000,000"
3,13/07/2020,Mystifly,Airfare Marketplace,Singapore and Bangalore,"Ticketing, Airline Retailing, and Post-Ticketi...",Recruit Co. Ltd.,pre-Series B,"3,300,000"
4,14/07/2020,goDutch,FinTech,Mumbai,Group Payments,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
5,10/07/2020,gigIndia,Marketplace,Pune,"Crowd Sourcing, Freelance",Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
6,16/07/2020,Crio,EduTech,Bangalore,Learning Platform for Developers,021 Capital,pre-Series A,"934,160"
7,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Bangalore,Internet-of-Things Security Solutions,Unicorn India Ventures,Venture-Series Unknown,"500,000"
8,09/07/2020,JetSynthesys,Gaming and Entertainment,Pune,Gaming and Entertainment,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
9,15/07/2020,PumPumPum,Automotive Rental,Gurgaon,Used Car-leasing platform,Early Adapters Syndicate,Seed,"292,800"


### 7. Write a program to scrap all the available details of best gaming laptops from digit.in. 

In [453]:
# accessing url
url = "https://www.digit.in/"
driver.get(url)
time.sleep(2)

#clicking on top 10
top_10=driver.find_element_by_xpath("/html/body/div[1]/div[2]/div[4]/ul/li[4]/a")
top_10.click()
time.sleep(2)

#clicking laprops
laptops=driver.find_element_by_xpath("/html/body/div[3]/div/div/div[2]/div[5]/div[1]/div/button[2]/img")
laptops.click()

#clicking on  the gaming laptops

best_gaming=driver.find_element_by_xpath("//div[@id='laptops']//div[3]//a")
driver.get(best_gaming.get_attribute('href'))

In [454]:
#creating lsits
name = []
Price = []
OS = []
display = []
processor = []
HDD = []
RAM = []
weight = []
dimension = []
GPU = []

time.sleep(1)
#names
names=driver.find_elements_by_xpath("//div[@class='right-container']/div/a/h3")
for i in names:
    name.append(i.text)

time.sleep(1)
#operating system
os=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[1]/div/div")
for i in os:
    OS.append(i.text)

time.sleep(1)
#scraping display
displays=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[2]/div/div")
for i in displays:
    display.append(i.text)

time.sleep(1)
# processor
processors=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[3]/div/div")
for i in processors:
    processor.append(i.text)

time.sleep(1)
# scraping memory
memories=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[3]")# extrat HDD and RAM form xpath
for i in memories:
    HDD.append(i.text.split("/")[0])
    RAM.append(i.text.split("/")[1])

time.sleep(1)
# scraping weight
weights=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[7]/td[3]")# extrat weight form xpath
for i in weights:
    weight.append(i.text)

time.sleep(1)
# scraping dimension
dimension=[]
dimensions=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[8]/td[3]") 
for i in dimensions:
    dimension.append(i.text)

time.sleep(1)
# scraping graphical processor
GPUs=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[9]/td[3]") 
for i in GPUs:
    GPU.append(i.text)

time.sleep(1)

#scraping price
price=driver.find_elements_by_xpath("//table[@id='summtable']//tr//td[3]")
for i in price:
    Price.append(i.text)

In [456]:
Best_gaming=pd.DataFrame({})
Best_gaming["Name"]=name
Best_gaming["price"]=Price
Best_gaming["OS"]=OS
Best_gaming["Display"]=display
Best_gaming["HDD"]=HDD
Best_gaming["RAM"]=RAM
Best_gaming["processor"]=processor
Best_gaming["weight"]=weight
Best_gaming["Dimension"]=dimension
Best_gaming["Graphical processor"]=GPU
Best_gaming

,Name,price,OS,Display,HDD,RAM,processor,weight,Dimension,Graphical processor
0,ALIENWARE AREA 51M R2,N/A,WINDOWS 10 HOME,"17.3"" (1920 X 1080)",1 TB SSD,16 GBGB DDR4,10TH GENERATION INTEL® CORE™ I7-10700 | 2.90 GHZ,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630
1,ALIENWARE M15 R3,₹341990,WINDOWS 10 HOME,"15.6"" (3840 X 2160)",1 TB SSD,16 GBGB DDR4,10TH GENERATION INTEL® CORE™ I9-10980HK | NA,NA,NA,NA
2,ASUS ROG STRIX SCAR 15,N/A,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",1 TB SSD,16 GBGB DDR4,AMD RYZEN™ 9 5900HX | 3.3 GHZ,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070
3,ASUS ROG ZEPHYRUS G14,₹164990,WINDOWS 10 HOME,"14"" (1920 X 1080)",1 TB SSD,16 GBGB DDR4,AMD 3RD GENERATION RYZEN 9 | 3.3 GHZ,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060
4,LENOVO LEGION 5I,₹71990,WINDOWS 10 PRO,"15.6"" (1920 X 1080)",1 TB SSD,16 GBGB DDR4,10TH GENERATION INTEL® CORE™ I5-10300H | 2.50 GHZ,2.3,363.06 x 259.61 x 23.57,NVIDIA® GeForce® GTX 1650 4GB
5,ASUS ROG ZEPHYRUS DUO 15,₹173800,WINDOWS 10,"15.6"" (3840 X 1100)",512 GB SSD,4 GBGB DDR4,INTEL CORE I7 10TH GEN 10875H | NA,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q
6,ACER ASPIRE 7 GAMING,₹56990,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",512 GB SSD,8 GBGB DDR4,AMD RYZEN™ 5-5500U HEXA-CORE | NA,2.15,2.29 x 36.3 x 25.4,NVIDIA® GeForce® GTX 1650


### 8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”,“Industry”.


In [601]:
#getting url
driver.get('https://www.forbes.com/billionaires/')

In [605]:
Rank=[]
Names=[]
Net_worth=[]
Age=[]
Citizenship=[]
Source=[]
Industry=[]
Rank_list=driver.find_elements_by_xpath("//div[@class='rank']")
Name_list=driver.find_elements_by_xpath("//div[@class='personName']")
worth_list=driver.find_elements_by_xpath("//div[@class='netWorth']")
age=driver.find_elements_by_xpath("//div[@class='age']")
source=driver.find_elements_by_xpath("//div[@class='source-column']")
country=driver.find_elements_by_xpath("//div[@class='countryOfCitizenship']")
industry=driver.find_elements_by_xpath("//div[@class='category']")
for name in Name_list:
    Names.append(name.text)
for rank in Rank_list:
    Rank.append(rank.text)
for worth in worth_list:
    Net_worth.append(worth.text)
for ag in age:
    Age.append(ag.text)
for src in source:
    Source.append(src.text)
for orgin in country:
    Citizenship.append(orgin.text)
for ind in industry:
    Industry.append(ind.text)
    


In [606]:
Billionares=pd.DataFrame({})
Billionares['Rank']=Rank
Billionares['Names']=Names
Billionares['Net_worth']=Net_worth
Billionares['Age']=Age
Billionares['Citizenship']=Citizenship
Billionares['Source']=Source
Billionares['Industry']=Industry
Billionares



,Rank,Names,Net_worth,Age,Citizenship,Source,Industry
0,1.,Jeff Bezos,$177 B,57,United States,Amazon,Technology
1,2.,Elon Musk,$151 B,49,United States,"Tesla, SpaceX",Automotive
2,3.,Bernard Arnault & family,$150 B,72,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$124 B,65,United States,Microsoft,Technology
4,5.,Mark Zuckerberg,$97 B,36,United States,Facebook,Technology
...,...,...,...,...,...,...,...
195,195.,Harry Triguboff,$11.2 B,88,Australia,real estate,Real Estate
196,197.,Leonid Fedun & family,$11.1 B,65,Russia,oil,Energy
197,197.,Eyal Ofer,$11.1 B,70,Israel,"real estate, shipping",Diversified
198,197.,Evan Spiegel,$11.1 B,30,United States,Snapchat,Technology


### 9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.


In [69]:
#accessing url
driver.get('https://www.youtube.com/watch?v=dFCbJmgeHmA')
time.sleep(3)
for i in range(180):
    time.sleep(5)
    driver.execute_script("window.scrollBy(0,350)") 

In [70]:
comments=[]
Upvotes=[]
Posted=[]


#getting comments,Likes and Date
comment_list=driver.find_elements_by_xpath("//yt-formatted-string[@class='style-scope ytd-comment-renderer']")

posted=driver.find_elements_by_xpath("//yt-formatted-string[@class='published-time-text above-comment style-scope ytd-comment-renderer']//a")

#comments to list
item=''
comments=[i.text.lstrip() for i in comment_list]

comments_n = [i for i in comments if i != item]

comments_n


#upvotes to list

#toolbar
toolbar=driver.find_elements_by_id("toolbar")
for i in toolbar:
    try:
        Likes=i.find_element_by_id("vote-count-middle")
        if Likes is None:
            Upvotes.append(0)
        else:
            Upvotes.append(Likes.text)   
        
    except NoSuchElementException:
        Upvotes.append(0)


#psted date to list
Posted=[j.text for j in posted]

In [81]:
#replacing empty values by 0
for n, i in enumerate(Upvotes):
 if i == '':
    Upvotes[n] = '0'
#cleaning comments
comments_n
comments_n=[i.replace('\n','') for i in comments_n]

In [103]:
Youtube=pd.DataFrame({})
Youtube['Comments']=comments_n[:500]
Youtube['Upvotes']=Upvotes[:500]
Youtube['Date_posted']=Posted[:500]
Youtube.head(5)

,Comments,Upvotes,Date_posted
0,Ohai! So a few more words about our announceme...,14K,2 weeks ago (edited)
1,"When he said ""Yeeted into space"" I had to rewi...",35K,2 weeks ago
2,It's frightening how an entire era disappeared...,1.4K,6 days ago
3,"The most scientific term you will EVER hear:""T...",1.2K,1 week ago
4,"”Long ago, the dinosaurs lived together in har...",415,1 week ago


### 10. Write a python program to scrape a data for all available Hostels from
https://www.hostelworld.com/ in “London” location. You have to scrape hostel name,
distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms
from price, facilities and property description. 

In [322]:
#getting url
driver.get('https://www.hostelworld.com/')
time.sleep(4)

#clicking the search field to enable locating the search bar element
random=driver.find_element_by_xpath("//div[@class='field body-2 location-input']")
random.click()
time.sleep(3)

#search_bar
search=driver.find_element_by_xpath("//input[@class='search-input']")
time.sleep(2)
search.send_keys('London')
time.sleep(2)

#selecting the first suggetion
select=driver.find_element_by_xpath('//*[@id="predicted-search-results"]/li[2]')
select.click()

#search_button- clicking twice to minimize the suggetions
search_btn=driver.find_element_by_id("search-button")
for i in range(1):
    search_btn.click()

In [323]:
#accessing and extracting page urls
page_urls=[]
##Name=[]
time.sleep(4)
for page in range(0,3):
    driver.execute_script("window.scrollBy(0,350)")
    page_urls.append(driver.current_url)
    time.sleep(5)
    try:
        next= driver.find_element_by_xpath("//i[@class='core-icon icon-core-chevron-right']")
        next.click()
    except:
         pass

In [436]:
Name=[]
distance=[]
Ratings=[]
Private=[]
Dorms=[]
Facility=[]
for url in page_urls:
    driver.get(url)
    time.sleep(2)
    
    #name of the hostel
    name=driver.find_elements_by_xpath("//h2[@class='title title-6']//a") #extracting the details frm each page
    for i in name:
        Name.append(i.text)
        
    #distance
    dist=driver.find_elements_by_xpath("//span[@class='description']")
    for d in dist:
        distance.append(d.text)
    
    #ratings
    try:
        rate=driver.find_elements_by_xpath("//div[@class='rating rating-summary-container big']")
        for r in rate:
            Ratings.append(r.text)
    except:
        rate=driver.find_elements_by_xpath("//div[@class='keyword']")
        for r in rate:
            Ratings.append(r.text)
            
    #private pricing
    private=driver.find_elements_by_xpath("//a[@class='prices']")
    for p in private:
            Private.append(p.text)
    facilities=driver.find_elements_by_xpath("//div[@class='facilities-label facilities']")
    for f in facilities:
            Facility.append(f.text.replace('\n',','))  

In [446]:
Numer_Rating=[]
Total_Ratings=[]
overall=[]
private_rooms=[]
dorm_rooms=[]
pvt=[]
description=[]
Distance=[]
#extracting the rating
for num in Ratings:       
    Numer_Rating.append(num.split("\n")[0])
    

#extracting total ratings
for tot in Ratings:
    Total_Ratings.append(tot.split('\n')[-1])
    
    
#extracting the overall ratings
rating=[i.split('\n') for i in Ratings]

for i in rating:
    if len(i)==2:
        overall.append('No rating')
    else:
        overall.append(i[1])
    
p_list=Private[3:]        #removing junk values
price=[i.split('\n') for i in p_list]

#Accessing the private room prices
for i in price:
    if len(i)==4:
        private_rooms.append(i[1])
    elif i[0]=="Privates From":
        private_rooms.append(i[1])
    else:
        private_rooms.append("No Private Rooms")

#Accessing the Dorm room prices
for j in price:
    if len(j)==4:
        dorm_rooms.append(j[3])
    elif j[1]=="Dorms From":
        dorm_rooms.append(j[2])
    else:
        dorm_rooms.append("No Dorm Rooms")
        
        
#extracting Description and distance
for i in distance:
    description.append(i.split('-')[0].strip())
for d in distance:
    Distance.append(d.split('-')[1].strip())

In [445]:
Hostels_London=pd.DataFrame({})
Hostels_London['Name']=Name
Hostels_London['Distance from city']=Distance
Hostels_London['Private_rooms']=private_rooms
Hostels_London['Dorm_rooms']=dorm_rooms
Hostels_London['Facility']=Facility
Hostels_London['Description']=description
Hostels_London['Rating']=Numer_Rating
Hostels_London['Reviews']=overall
Hostels_London['total reviews']=Total_Ratings
Hostels_London

,Name,Distance from city,Private_rooms,Dorm_rooms,Facility,Description,Rating,Reviews,total reviews
0,St Christopher's Village,1.8km from city centre,Rs3408,Rs1435,"Free WiFi,Follows Covid-19 sanitation guidance",Hostel,8.9,Fabulous,10814 Total Reviews
1,Generator London,3km from city centre,Rs8571,Rs2169,"Free WiFi,Follows Covid-19 sanitation guidance",Hostel,7.5,Very Good,6729 Total Reviews
2,Safestay London Kensington Holland Park,5.9km from city centre,Rs4180,Rs1022,Free WiFi,Hostel,7.9,Very Good,1070 Total Reviews
3,"PubLove @ The Crown, Battersea",4.7km from city centre,No Private Rooms,Rs1239,"Free WiFi,Follows Covid-19 sanitation guidance",Hostel,7.9,Very Good,204 Total Reviews
4,Leman House,3.6km from city centre,Rs5577,No Dorm Rooms,Free WiFi,Hostel,No Rating,No rating,13 Total Reviews
...,...,...,...,...,...,...,...,...,...
80,The Dover,1.9km from city centre,Rs6747,No Dorm Rooms,"Free WiFi,Free Breakfast",Hotel,No Rating,No rating,4 Total Reviews
81,Park Hotel Essex,24.1km from city centre,Rs3752,No Dorm Rooms,"Free Breakfast,Follows Covid-19 sanitation gui...",Hotel,No Rating,No rating,108 Total Reviews
82,Cranbrook Hotel,14.8km from city centre,Rs3098,No Dorm Rooms,"Free Breakfast,Follows Covid-19 sanitation gui...",Hotel,No Rating,No rating,58 Total Reviews
83,St. Athans,2.9km from city centre,Rs3976,No Dorm Rooms,"Free WiFi,Follows Covid-19 sanitation guidance",Bed and Breakfast,No Rating,No rating,234 Total Reviews
